# **Setup**

In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install --quiet cornac==1.15.4 adjustText

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 19.7 MB/s eta 0:00:00


In [20]:
import os
import sys
import itertools

import scipy.sparse as sp
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from adjustText import adjust_text
%matplotlib inline

import cornac
from cornac.data import Reader
#from cornac.utils import cache
#from cornac.data import TextModality
from cornac.datasets import movielens
from cornac.datasets import amazon_clothing
from cornac.eval_methods import RatioSplit
from cornac.models import MF, NMF, BaselineOnly, CTR, WMF
from cornac.data.text import BaseTokenizer

print(f"System version: {sys.version}")
print(f"Cornac version: {cornac.__version__}")

SEED = 42
VERBOSE = True
MAX_ITER=20
K_DIM=15

System version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0]
Cornac version: 1.15.4


In [4]:
anime_path="/content/drive/MyDrive/RecSys/amime/rating_complete.csv"
#anime_path="/home/kgamegah/Documents/academic/UPC/RecommenderSystems/RecSys/amime/rating_complete.csv"

In [5]:
data = Reader().read(fpath=anime_path, fmt='UIR', sep=',', skip_lines=1)
#data

In [6]:
#from cornac.data import Dataset
# Load the data into a dataset object
#dataset = Dataset.from_uir(data)

# **NMF**

In [7]:
# Instantiate an evaluation method.
eval_method = RatioSplit(
    data=data,
    test_size=0.2,
    #rating_threshold=4.0,
    #exclude_unknowns=True,
    verbose=VERBOSE,
    seed=SEED,
)

In [22]:
# Instantiate a NMF recommender model.
nmf = cornac.models.NMF(
    k=K_DIM,
    max_iter=MAX_ITER,
    learning_rate=0.01,
    lambda_reg=0.0,
    #lambda_u=0.06,
    #lambda_v=0.06,
    #lambda_bu=0.02,
    #lambda_bi=0.02,
    use_bias=False,
    verbose=VERBOSE,
    seed=SEED,
    name=f"NMF(K={K_DIM})"
)

mf = cornac.models.MF(
    k=K_DIM,
    max_iter=MAX_ITER,
    learning_rate=0.01,
    lambda_reg=0.0,
    use_bias=False,
    verbose=VERBOSE,
    seed=SEED,
    name=f"MF(K={K_DIM})")

In [23]:
# Instantiate evaluation metrics.
mae = cornac.metrics.MAE()
rmse = cornac.metrics.RMSE()
rec_20 = cornac.metrics.Recall(k=20)
pre_20 = cornac.metrics.Precision(k=20)

In [24]:
# Instantiate and then run an experiment.
cornac.Experiment(
    eval_method=eval_method,
    models=[nmf, mf],
    metrics=[
        mae,
        rmse,
        rec_20,
        pre_20
        ]
).run()

  0%|          | 0/20 [00:00<?, ?it/s]

Optimization finished!


  0%|          | 0/20 [00:00<?, ?it/s]

Optimization finished!

TEST:
...
          |    MAE |   RMSE | Precision@20 | Recall@20 | Train (s) |  Test (s)
--------- + ------ + ------ + ------------ + --------- + --------- + ---------
NMF(K=15) | 2.8345 | 3.0340 |       0.0136 |    0.0093 |   78.3526 | 1092.6251
MF(K=15)  | 0.9038 | 1.1280 |       0.0399 |    0.0351 |  108.9892 | 1109.6677
